In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
stop_words = set(stopwords.words('english')) 
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Input
from keras.models import Model
from keras import optimizers
from keras.layers import concatenate,CuDNNLSTM
from keras import initializers
from keras import backend as K
import random

from keras.layers import Dense, LSTM,Dropout, Activation,Bidirectional,Reshape,Permute,Multiply,Flatten,Lambda,Layer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Using TensorFlow backend.


In [0]:



'''
Breaking whole data into list of sentences
'''
def seperate_sentences(data):
    return sent_tokenize(data) 


def Sentence_to_POS(all_sentences):

    data=[]
    for sentence in all_sentences: 
        one_sentence_pos = []
        '''
        Word tokenizers is used to find the words and punctuation in a string 
        '''  
        wordsList = nltk.word_tokenize(sentence) 

        '''
        Removing stop words from wordList.
        ''' 
        wordsList = [w for w in wordsList if not w in stop_words]  

        '''
        Using a Tagger. Which is part-of-speech tagger or POS-tagger.  
        '''

        tagged = nltk.pos_tag(wordsList) 
        for x in tagged:
            one_sentence_pos.append(x[1])
        data.append(one_sentence_pos)

    return data


def Fianl_X_and_Y(novel_POS,novel_label,number_of_sentenes_in_one):
    '''
    This funtion takes generates more data for training by breaking one novel into multiple novels
    with k number of sentences (number_of_sentenes_in_one) .we are ignoring some last sentences if they
    dont fit
    '''
    i=0
    X=[]
    Y=[]
    for novel in novel_POS:
        start = 0
        end = number_of_sentenes_in_one
        while(end<=len(novel)):
            X.append(novel[start:end])
            Y.append(novel_label[i])
            start=end
            end=end+number_of_sentenes_in_one
        i+=1
    return X,Y


def Get_X_and_Y_In_POS_Form(path_of_dataset_directory,number_of_sentences_in_one):
    '''
    this function takes the path of directory where the dataset is and returns the processed X and Y.
    The function calls will let u understand more the flow of code
    '''
#     all_files = os.listdir(path_of_dataset_directory)
    all_files = ['12ItrainA1.TXT','12ItrainA2.TXT','12ItrainB1.TXT','12ItrainB2.TXT','12ItrainC1.TXT','12ItrainC2.TXT','12ItrainD1.TXT','12ItrainD2.TXT'
                ,'12ItrainE1.TXT','12ItrainE2.TXT','12ItrainF1.TXT','12ItrainF2.TXT','12ItrainG1.TXT','12ItrainG2.TXT','12ItrainH1.TXT','12ItrainH2.TXT'
                ,'12ItrainI1.TXT','12ItrainI2.TXT','12ItrainJ1.TXT','12ItrainJ2.TXT','12ItrainK1.TXT','12ItrainK2.TXT','12ItrainL1.TXT','12ItrainL2.TXT'
                 ,'12ItrainM1.TXT','12ItrainM2.TXT','12ItrainN1.TXT','12ItrainN2.TXT']
    novel_POS=[]
    novel_label=[]
    file_number=1
    for filename in all_files:
        print(file_number,"/",len(all_files)," Done")
        file_number+=1
        full_path = '/content/gdrive/My Drive/dataset_smai/'+filename
        with open(full_path, 'r') as f:
            data = f.read().replace('"\n"','').replace('\n',' ').replace('- ','')
            all_sentences = seperate_sentences(data)
            sentences_to_pos = Sentence_to_POS(all_sentences)
            novel_POS.append(sentences_to_pos)
            novel_label.append(filename[8])
    return Fianl_X_and_Y(novel_POS,novel_label,number_of_sentences_in_one)

def tag_to_index_dictionary(X):
    '''
    This function just creates a dictionary from the POS tags which we
    encountered in our dataset.As the network works with numbers so simple will make
    dictionary which stores the index of associated POS tag
    '''
    tag = set([])
    for doc in X:
        for sentence in doc:
            for word in sentence:
                tag.add(word)
    tag2index = {t: i + 1 for i, t in enumerate(list(tag))}
    tag2index['-PAD-'] = 0
    return tag2index

def convert_tag_to_sequence_numbers(X):
    tag2index = tag_to_index_dictionary(X)
    '''
    using the tag2index dictionary assign indexs to the POS tags in our data
    '''
    new_X=[]
    for doc in X:
        new_S = []
        for sentence in doc:
            new_W=[]
            for word in sentence:
                new_W.append(tag2index[word])
            new_S.append(new_W)
        new_X.append(new_S)
    return new_X


def pad_zeros_to_sequence(X,max_length):
    '''
    This function padds zeros to the sequences so as to make fixed sequences
    According to paper 15 is best value for max_length
    '''
    
    new_X=[]
    for doc in X:
        new_X.append(pad_sequences(doc, maxlen=max_length, padding='post'))
    return new_X

def Encode_Labels(Y):
    '''
    This fucntion encodes the labels i.e., assign numbers to the labels
    '''
    le = LabelEncoder()
    return le.fit_transform(Y)




In [4]:
print("Preprocessing Phase 1 : Fetching data")
print(".....................................")

X,Y = Get_X_and_Y_In_POS_Form('dataset/three_author_dataset',100)

print("Preprocessing Phase 1 : Finished\n\n")
print(".....................................")

Preprocessing Phase 1 : Fetching data
.....................................
1 / 28  Done
2 / 28  Done
3 / 28  Done
4 / 28  Done
5 / 28  Done
6 / 28  Done
7 / 28  Done
8 / 28  Done
9 / 28  Done
10 / 28  Done
11 / 28  Done
12 / 28  Done
13 / 28  Done
14 / 28  Done
15 / 28  Done
16 / 28  Done
17 / 28  Done
18 / 28  Done
19 / 28  Done
20 / 28  Done
21 / 28  Done
22 / 28  Done
23 / 28  Done
24 / 28  Done
25 / 28  Done
26 / 28  Done
27 / 28  Done
28 / 28  Done
Preprocessing Phase 1 : Finished


.....................................


In [5]:
np.unique(Y,return_counts=True)

(array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
        'N'], dtype='<U1'),
 array([ 95, 264, 191, 125, 180,  72, 100, 159,  89, 104,  79,  47,  60,
         74]))

In [0]:
def Balance_Data(X,Y):
  MIN_VAL = min(np.unique(Y,return_counts=True)[1])+200
  A,B,C,D,E,F,G,H,I,J,K,L,M,N=0,0,0,0,0,0,0,0,0,0,0,0,0,0
  X_data=[]
  for i in range(len(X)):
      if(Y[i]=='A' and A<MIN_VAL):
          X_data.append([X[i],Y[i]])
          A+=1
      elif(Y[i]=='B' and B<MIN_VAL):
          X_data.append([X[i],Y[i]])
          B+=1
      elif(Y[i]=='C' and C<MIN_VAL):
          X_data.append([X[i],Y[i]])
          C+=1

      elif(Y[i]=='D' and D<MIN_VAL):
          X_data.append([X[i],Y[i]])
          D+=1

      elif(Y[i]=='E' and E<MIN_VAL):
          X_data.append([X[i],Y[i]])
          E+=1

      elif(Y[i]=='F' and F<MIN_VAL):
          X_data.append([X[i],Y[i]])
          F+=1

      elif(Y[i]=='G' and G<MIN_VAL):
          X_data.append([X[i],Y[i]])
          G+=1

      elif(Y[i]=='H' and H<MIN_VAL):
          X_data.append([X[i],Y[i]])
          H+=1

      elif(Y[i]=='I' and I<MIN_VAL):
          X_data.append([X[i],Y[i]])
          I+=1

      elif(Y[i]=='J' and J<MIN_VAL):
          X_data.append([X[i],Y[i]])
          J+=1

      elif(Y[i]=='K' and K<MIN_VAL):
          X_data.append([X[i],Y[i]])
          K+=1

      elif(Y[i]=='L' and L<MIN_VAL):
          X_data.append([X[i],Y[i]])
          L+=1

      elif(Y[i]=='M' and M<MIN_VAL):
          X_data.append([X[i],Y[i]])
          M+=1

      elif(Y[i]=='N' and N<MIN_VAL):
          X_data.append([X[i],Y[i]])
          N+=1
    
  random.shuffle(X_data)
        
  new_X=[]
  new_Y=[]
  for m , n in X_data:
    new_X.append(m)
    new_Y.append(n)
    
  return new_X,new_Y

X,Y=Balance_Data(X,Y)

In [0]:
DICTIONARY_LENGTH = len(tag_to_index_dictionary(X))
MAX_LENGTH = 30
SENTENCES_IN_NOVEL=100

X = convert_tag_to_sequence_numbers(X)
X = pad_zeros_to_sequence(X,MAX_LENGTH)
X = np.array(X)
TOTAL_ROWS=X.shape[0]

In [8]:
TOTAL_ROWS

1622

In [0]:
real_X = []
for i in range(SENTENCES_IN_NOVEL):
    real_X.append(list())
for i in range(SENTENCES_IN_NOVEL):
    for j in range(TOTAL_ROWS):
        real_X[i].append(X[j][i])

X_train = []
for i in range(SENTENCES_IN_NOVEL):
    X_train.append(np.array(real_X[i]))



Y = Encode_Labels(Y) 
from tensorflow.keras.utils import to_categorical
y_binary = to_categorical(Y)


In [10]:
np.array(real_X).shape

(100, 1622, 30)

In [0]:
from keras.engine.topology import Layer as LL
class Attention(LL):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        print(self.W_constraint)
        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
        print(eij.shape,self.b.shape)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [12]:
print("s")

s


In [13]:
from keras import  regularizers,constraints,initializers
print("Preprocessing Phase 2 : Finished\n\n")
print(".....................................")
print("Shape of Actual X ",np.array(real_X).shape)
print("Shape of Changed X ",np.array(real_X).shape)
print("Shape of Y" ,y_binary.shape)
print(".....................................")
print("\n\nMode Construction")
print(".....................")

# SENTENCES_IN_NOVEL=5
# MAX_LENGTH=30
# DICTIONARY_LENGTH=47

FIRST_LSTM =256
SECOND_LSTM=256
EMBEDD_VECTOR_LENGTH=150
outputs=[]
inputs_=[]

for i in range(SENTENCES_IN_NOVEL):# SENTENCES_IN_NOVEL = 100
    inputlayer = Input(shape=[MAX_LENGTH]) #MAX_LENGTH = 30 (30 words per sentence)
    inputs_.append(inputlayer)
    layer = Embedding(DICTIONARY_LENGTH, EMBEDD_VECTOR_LENGTH, input_length=30)(inputlayer) # 50 is embedd vector size for each POS Tag
    layer =  Bidirectional(LSTM(FIRST_LSTM,return_sequences=True),merge_mode='concat')(layer)#output 100 because of bidirectonal concat
    layer =  Lambda( lambda x: K.sum(x, axis=1), input_shape=(30,2*FIRST_LSTM))(layer)
    outputs.append(layer)
    
merge_ = concatenate(outputs)
merge_ = Reshape((SENTENCES_IN_NOVEL, 2*FIRST_LSTM), input_shape=(SENTENCES_IN_NOVEL*2*FIRST_LSTM,))(merge_)
merge_ = Bidirectional(LSTM(SECOND_LSTM,return_sequences=True),merge_mode='concat')(merge_)


attention_mul = Attention(100)(merge_)#step_dimen =100
output = Dense(14, activation='softmax')(attention_mul
                                        )

model = Model(inputs=inputs_,outputs=output)
model.summary()

Preprocessing Phase 2 : Finished


.....................................
Shape of Actual X  (100, 1622, 30)
Shape of Changed X  (100, 1622, 30)
Shape of Y (1622, 14)
.....................................


Mode Construction
.....................
Instructions for updating:
Colocations handled automatically by placer.
None
(?, 100) (100,)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 30)           0             

In [14]:
 np.array([1,2,3,4,5,6]).reshape(2,3)

array([[1, 2, 3],
       [4, 5, 6]])

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Nadam(lr=0.001),
              metrics=['accuracy'])
model.fit(X_train,y_binary,epochs=40,validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 1459 samples, validate on 163 samples
Epoch 1/40
1459/1459 [==============================] - 2028s 1s/step - loss: 2.4838 - acc: 0.1638 - val_loss: 2.3297 - val_acc: 0.2209
Epoch 2/40
1459/1459 [==============================] - 1630s 1s/step - loss: 2.0482 - acc: 0.2906 - val_loss: 1.9666 - val_acc: 0.3129
Epoch 3/40
1459/1459 [==============================] - 1709s 1s/step - loss: 1.7269 - acc: 0.3639 - val_loss: 1.7171 - val_acc: 0.3988
Epoch 4/40
  32/1459 [..............................] - ETA: 27:26 - loss: 1.8115 - acc: 0.3125